In [ ]:
"""analisis de malware y benigno con dataset y SVM
preprocesado por chinos con binary
paper 244802 en df 293333"""







import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix 
from sklearn import datasets, metrics
from sklearn.metrics import classification_report 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import os
import glob

"""matplotlib inline"""
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob

# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
os.chdir("..\\Datasets")
os.getcwd()
csv_files = glob.glob('*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes

for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)

#Para chequear que todo está bien, mostramos la list_data por consola
#list_data
 
df = pd.concat(list_data,ignore_index=True)

tipos=df.dtypes  #tipos de datos de campos

"""información datos"""
print(df.head(5))
print('Cantidad de Filas y columnas:',df.shape)
print('Nombre columnas:',df.columns)

"""Columnas, nulos y tipo de datos"""
print(df.info())

"""descripción estadística de los datos numéricos"""
print(df.describe())

"""correlación entre los datos"""
corr = df.set_index('Label').corr()
#sm.graphics.plot_corr(corr, xnames=list(corr.columns))
#plt.show()

df2=df[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']]
df3=df[['Label']]

"""convertir columna tipo dato str a int"""
df2['Label']=df3[['Label']]

"""desordena el dataset"""

df2=df2.sort_values('Fwd_Pkt_Len_Min')
df2 = df2[df2['Label'].isin([1,2,3,4])]

""" convertir valores número la clase"""
df2['Label'] = df2['Label'].replace(1,"Benigno")
df2['Label'] = df2['Label'].replace(2,"Malware")
df2['Label'] = df2['Label'].replace(3,"Malware")
df2['Label'] = df2['Label'].replace(4,"Malware")
#print(df2.head(10))
#print(df2.groupby('Label').size())


dff=df2[:]
dff1 = dff[dff.Label == 'Malware']
dff = dff.drop(dff[dff['Label'] == 'Malware'].index)
a = dff1.iloc[0:134800,]
df2 = pd.concat([dff,a])



print(df2.groupby('Label').size())



  
# Split dataset in training and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(df2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']],df2['Label'], test_size=0.2, random_state=6) 

# training a linear SVM classifier 
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel='rbf', random_state=6, gamma=.01, C=1).fit(X_train, Y_train) 
svm_predictions = svm_model_linear.predict(X_test) 
  
# model accuracy for X_test   
accuracy = svm_model_linear.score(X_train, Y_train)*100
print("precisión entrenamiento : " + str(accuracy))
accuracy = svm_model_linear.score(X_test, Y_test)*100
print("precisión validado : " + str(accuracy))
# creating a confusion matrix 
cm = confusion_matrix(Y_test, svm_predictions) 
print(cm)
print(classification_report(Y_test, svm_predictions))

""" convierto valor nombre a 0=benigno 1=malware para AUC-ROC"""

label_encoder = LabelEncoder()
df2.iloc[:,15] = label_encoder.fit_transform(df2.iloc[:,15]).astype('int64')
X_train, X_test, Y_train, Y_test = train_test_split(df2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']],df2['Label'], test_size=0.2, random_state=6) 


def plot_roc(model, X_test, Y_test):
    # calculate the fpr and tpr for all thresholds of the classification
    probabilities = model.predict_proba(np.array(X_test))
    predictions = probabilities[:, 1]
    fpr, tpr, threshold = metrics.roc_curve(Y_test, predictions)
    roc_auc = metrics.auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

plot_roc(svm_model_linear, X_test, Y_test)


"""============================================="""
""" CROSS VALIDATION"""

X_trainCV, X_testCV, Y_trainCV, Y_testCV = train_test_split(df2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']],df2['Label'], test_size=0.2, random_state=6) 

clfCV = SVC(kernel='rbf', random_state=6, gamma=.01, C=1).fit(X_train, Y_train) 

kf = KFold(n_splits=10)
score = clfCV.score(X_trainCV,Y_trainCV)
print("Metrica del modelo", score * 100)
scores = cross_val_score(clfCV, X_trainCV, Y_trainCV, cv=kf, scoring="accuracy")
print("Metricas cross_validation", scores * 100)
#print("Media de cross_validation", scores.mean() * 100)
preds = clfCV.predict(X_testCV)
score_pred = metrics.accuracy_score(Y_testCV, preds)
print("Metrica en Test", score_pred * 100)
conf_mat = confusion_matrix(Y_testCV, preds)
print(conf_mat)
print(classification_report(Y_testCV, preds))

def plot_roc(model, X_test, Y_test):
    # calculate the fpr and tpr for all thresholds of the classification
    probabilities = model.predict_proba(np.array(X_test))
    predictions = probabilities[:, 1]
    fpr, tpr, threshold = metrics.roc_curve(Y_test, predictions)
    roc_auc = metrics.auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

plot_roc(clfCV, X_testCV, Y_testCV)



"""
con
Label
Benigno    137907
Malware    155426
dtype: int64
precisión entrenamiento : 86.8472637706357
precisión validado : 86.86996096613088
[[23347  4223]
 [ 3480 27617]]
              precision    recall  f1-score   support
     Benigno       0.87      0.85      0.86     27570
     Malware       0.87      0.89      0.88     31097
    accuracy                           0.87     58667
   macro avg       0.87      0.87      0.87     58667
weighted avg       0.87      0.87      0.87     58667
sin
[8 rows x 16 columns]
Label
Benigno    137907
Malware    155426
dtype: int64
precisión entrenamiento : 99.58749882812167
precisión validado : 82.26260078067739
[[17227 10235]
 [  171 31034]]
              precision    recall  f1-score   support
     Benigno       0.99      0.63      0.77     27462
     Malware       0.75      0.99      0.86     31205
    accuracy                           0.82     58667
   macro avg       0.87      0.81      0.81     58667
weighted avg       0.86      0.82      0.82     58667
"""

['benign-2c .csv', 'malware.csv']
   Flow_Duration  Tot_Fwd_Pkts  Tot_Bwd_Pkts  TotLen_Fwd_Pkts  \
0           5870             2             0              813   
1           1499             2             0              128   
2             16             0             2                0   
3          46435             1             1               79   
4              0             1             0              356   

   TotLen_Bwd_Pkts  Fwd_Pkt_Len_Min  Bwd_Pkt_Len_Min  Fwd_Pkt_Len_Max  \
0                0               83             -1.0              730   
1                0               64             -1.0               64   
2              159               -1             52.0               -1   
3               95               79             95.0               79   
4                0              356             -1.0              356   

   Bwd_Pkt_Len_Max  Fwd_Pkt_Len_Mean  Bwd_Pkt_Len_Mean  Fwd_Pkt_Len_Std  \
0               -1             406.5               0.0       